In [0]:
import numpy as np
import pyopencl as cl
import time
import os
import math
from pyopencl import array
from time import time

def matrix_log10(queue,ctx,prg,a_np,res_np):
 mf = cl.mem_flags
 t1 = time()
 inp = a_np 
 a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
 _knl = prg.matrix_log10
 _knl.set_scalar_arg_dtypes([None,np.uint32])
 _knl(queue,a_np.shape, None,a_g,a_np.shape[0])
 cl.enqueue_copy(queue,res_np, a_g)
 push_time = time()-t1

 print('matrix_log10\n\n')
 print('Input Matrix\n')
 print(inp)
 print('\n')
 print('GPU Matrix \n')
 print(res_np)
 print('\n')
 print('CPU Matrix \n') 
 t1 = time()
 res_p = np.log10(a_np)
 cpu_time = time()-t1
 print(res_p)
 print('\n')
 if (push_time<cpu_time):
  print('GPU execution is faster')
 else:
  print('CPU execution is faster')

def matrix_minus_scalar(queue,ctx,prg,a_np,res_np,scalar):
 mf = cl.mem_flags
 t1 = time()
 inp = a_np 
 a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
 _knl = prg.matrix_minus_scalar
 _knl.set_scalar_arg_dtypes([None,np.uint32,np.double])
 _knl(queue,a_np.shape, None,a_g,a_np.shape[0],scalar)
 cl.enqueue_copy(queue,res_np, a_g)
 push_time = time()-t1

 print('matrix_minus_scalar\n\n')
 print('Input Matrix\n')
 print(inp)
 print('\n')
 print('GPU Matrix \n')
 print(res_np)
 print('\n')
 print('CPU Matrix \n') 
 t1 = time()
 res_p = a_np-scalar
 cpu_time = time()-t1
 print(res_p)
 print('\n')
 if (push_time<cpu_time):
  print('GPU execution is faster')
 else:
  print('CPU execution is faster')

def matrix_minus_matrix(queue,ctx,prg,a_np,b_np,res_np):
 mf = cl.mem_flags
 t1 = time() 
 a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
 b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)
 res_g = cl.Buffer(ctx, mf.WRITE_ONLY, res_np.nbytes)
 _knl = prg.matrix_minus_matrix
 _knl.set_scalar_arg_dtypes([None,np.uint32,None,None])
 _knl(queue,res_np.shape, None,a_g,a_np.shape[0],b_g,res_g)
 cl.enqueue_copy(queue, res_np, res_g)
 push_time = time()-t1
 print('matrix_minus_matrix\n\n')
 print('Input Matrix\n')
 print(a_np)
 print(b_np)
 print('\n')
 print('GPU Matrix \n')
 print(res_np)
 print('\n')
 print('CPU Matrix \n') 
 t1 = time()
 res_p = a_np-b_np
 cpu_time = time()-t1
 print(res_p)
 
 print('\n')
 if (push_time<cpu_time):
  print('GPU execution is faster\n')
 else:
  print('CPU execution is faster\n')

def matrix_multiply_col_vector(queue,ctx,prg,a_np,vec,M):
 mf = cl.mem_flags
 res_np = np.zeros((M,1),np.float32)
 t1 = time() 
 a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
 vec_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vec)
 res_g = cl.Buffer(ctx, mf.WRITE_ONLY, res_np.nbytes)
 _knl = prg.vector_mul
 _knl(queue,res_np.shape, None,a_g,vec_g,res_g)
 cl.enqueue_copy(queue, res_np, res_g)
 push_time = time()-t1
 print('matrix_multiply_col_vector\n\n')
 print('Input Matrix\n')
 print(a_np)
 print(vec)
 print('\n')
 print('GPU Matrix \n')
 print(res_np)
 print('\n')
 print('CPU Matrix \n') 
 t1 = time()
 res_p = np.dot(a_np,vec)
 cpu_time = time()-t1
 print(res_p)
 
 print('\n')
 if (push_time<cpu_time):
  print('GPU execution is faster\n')
 else:
  print('CPU execution is faster\n')

def matrix_divide_matrix(queue,ctx,prg,a_np,b_np,res_np):
 mf = cl.mem_flags
 t1 = time() 
 a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
 b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)
 res_g = cl.Buffer(ctx, mf.WRITE_ONLY, res_np.nbytes)
 _knl = prg.matrix_minus_matrix
 _knl.set_scalar_arg_dtypes([None,np.uint32,None,None])
 _knl(queue,res_np.shape, None,a_g,a_np.shape[0],b_g,res_g)
 cl.enqueue_copy(queue, res_np, res_g)
 push_time = time()-t1
 print('matrix_divide_matrix\n\n')
 print('Input Matrix\n')
 print(a_np)
 print(b_np)
 print('\n')
 print('GPU Matrix \n')
 print(res_np)
 print('\n')
 print('CPU Matrix \n') 
 t1 = time()
 res_p = a_np/b_np
 cpu_time = time()-t1
 print(res_p)
 
 print('\n')
 if (push_time<cpu_time):
  print('GPU execution is faster\n')
 else:
  print('CPU execution is faster\n')

def matrix_transpose(queue,ctx,prg,a_np,res_np):
 mf = cl.mem_flags
 BLOCK_DIM = 16
 t1 = time() 
 a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
 res_g = cl.Buffer(ctx, mf.WRITE_ONLY, res_np.nbytes)
 _knl = prg.matrix_transpose
 _knl.set_scalar_arg_dtypes([None,None,np.uint32,np.uint32,None])
 _knl(queue,res_np.shape,None,a_g,res_g,a_np.shape[1],a_np.shape[0],cl.LocalMemory(4*BLOCK_DIM*(BLOCK_DIM+1)))
 cl.enqueue_copy(queue, res_np, res_g)
 push_time = time()-t1
 print('matrix_divide_matrix\n\n')
 print('Input Matrix\n')
 print(a_np)
 print('\n')
 print('GPU Matrix \n')
 print(res_np)
 print('\n')
 print('CPU Matrix \n') 
 t1 = time()
 tran = [[a_np[j][i] for j in range(len(a_np))] for i in range(len(a_np[0]))] 
 cpu_time = time() - t1
 print("\n") 
 
 for item in tran: 
    print(item)
 
 print('\n')
 if (push_time<cpu_time):
  print('GPU execution is faster')
 else:
  print('CPU execution is faster')

def main():
 N = 9999
 M = 9999
 a_np = np.random.rand(M,N).astype(np.float64)
 b_np = np.random.rand(M,N).astype(np.float64)
 res_np = np.zeros((M,N),np.float64)
 mat_np = np.random.rand(M,N).astype(np.float32)
 vec = np.random.rand(N,1).astype(np.float32)
 scalar = 2
 ctx = cl.create_some_context()
 queue = cl.CommandQueue(ctx)
 prg = cl.Program(ctx, """
	 __kernel void matrix_log10
		(
			__global double * matrix,	
			uint row_size			
		)
	{
		// gid0 - numer wiersza macierzy input
		uint gid0 = get_global_id(0);
		// gid1 - numer elementu w wierszu.
		uint gid1 = get_global_id(1);

		if(gid1 >= row_size)
		{
			return;
		}
	
		uint idx = gid0 * row_size + gid1;

		double m = matrix[idx];
		m = log10(m);
		matrix[idx] = m;
	}

	__kernel void matrix_minus_scalar
		(
			__global double * matrix,	
			uint row_size,			
			double subtrahend		
		)
	{
		// gid0 - numer wiersza macierzy input
		uint gid0 = get_global_id(0);
		// gid1 - numer elementu w wierszu.
		uint gid1 = get_global_id(1);

		if(gid1 >= row_size)
		{
			return;
		}
	
		uint idx = gid0 * row_size + gid1;

		double m = matrix[idx];
		m -= subtrahend;
		matrix[idx] = m;
	}

	__kernel void matrix_minus_matrix
		(
			__global double * matrix,	
			uint row_size,			
			__global double * subtrahend_matrix,		
			__global double * output_matrix	
		)
	{
		// gid0 - numer wiersza macierzy input
		uint gid0 = get_global_id(0);
		// gid1 - numer elementu w wierszu.
		uint gid1 = get_global_id(1);

		if(gid1 >= row_size)
		{
			return;
		}
	
		uint idx = gid0 * row_size + gid1;

		double value = matrix[idx];
		double subtrahend = subtrahend_matrix[idx];

		value -= subtrahend;

		output_matrix[idx] = value;
	}

	__kernel void matrix_divide_matrix
		(
			__global double * dividend_matrix,	
			uint row_size,				
			__global double * divisor_matrix,	
			__global double * output_matrix		
		)
	{
		// gid0 - numer wiersza macierzy input
		uint gid0 = get_global_id(0);
		// gid1 - numer elementu w wierszu.
		uint gid1 = get_global_id(1);

		if(gid1 >= row_size)
		{
			return;
		}
	
		uint idx = gid0 * row_size + gid1;

		double dividend = dividend_matrix[idx];
		double divisor = divisor_matrix[idx];

		output_matrix[idx] = dividend /= divisor;
	}

	// Mnoży każdy element z i-tej kolumny przez i-ty element
	// podanego wektora.
	//
	__kernel void matrix_multiply_col_vector
		(
			__global double * matrix,	
			uint row_size,			
			__constant double * vector,	
							
			__global double * output_matrix	
		)
	{
		// gid0 - numer wiersza macierzy input
		uint gid0 = get_global_id(0);
		// gid1 - numer elementu w wierszu (numer kolumny).
		uint gid1 = get_global_id(1);

		if(gid1 >= row_size)
		{
			return;
		}
	
		uint idx = gid0 * row_size + gid1;
		uint col_idx = gid1;

		double value = matrix[idx];
		double m = vector[col_idx];

		value *= m;

		output_matrix[idx] = value;
	}

	__kernel void vector_mul(
	    __global const float4 *a_g, __global const float4 *b_g, __global float *res_g)
	{
	  int gid = get_global_id(0);
	  res_g[gid] = dot(a_g[gid], b_g[0]);
	}

	#define M_TRANSPOSE_BLOCK_DIM 16

	__kernel void matrix_transpose
		(
			__global double * matrix,	
			__global double * tmatrix,	
			uint width,			
			uint height,			
			__local double * scratch
		)
	{
		// gid0 - numer wiersza
		uint x_idx = get_global_id(0);
		// gid1 - numer kolumny
		uint y_idx = get_global_id(1);
	
		uint idx;

		// Pobieranie wartości z matrix	
		if((x_idx < width) && (y_idx < height))
		{	
			idx = y_idx * width + x_idx;
			scratch[get_local_id(1)*(M_TRANSPOSE_BLOCK_DIM+1)+get_local_id(0)] = matrix[idx];
		}
		barrier(CLK_LOCAL_MEM_FENCE);


		// Pobieranie wartości z matrix	
		x_idx = get_group_id(1) * M_TRANSPOSE_BLOCK_DIM + get_local_id(0);
		y_idx = get_group_id(0) * M_TRANSPOSE_BLOCK_DIM + get_local_id(1);
		if((x_idx < height) && (y_idx < width))
		{	
			idx = y_idx * height + x_idx;
			tmatrix[idx] = scratch[get_local_id(0)*(M_TRANSPOSE_BLOCK_DIM+1)+get_local_id(1)];
		}
	}""").build()
 if(0):
  matrix_log10(queue,ctx,prg,a_np,res_np)
 if(0):
  matrix_minus_matrix(queue,ctx,prg,a_np,b_np,res_np)
 if(1):
  matrix_minus_scalar(queue,ctx,prg,a_np,res_np,scalar)
 if(0):
  matrix_multiply_col_vector(queue,ctx,prg,mat_np,vec,M)
 if(0):
  matrix_divide_matrix(queue,ctx,prg,a_np,b_np,res_np)
 if(0):
  matrix_transpose(queue,ctx,prg,a_np,res_np)
 
if __name__ == '__main__':
 main()